In [26]:
import calendar
import datetime
import numpy as np
import os
import pandas as pd
import urllib.request
from urllib.request import urlopen
import plotly.express as px
import json
import requests
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns
import calplot
import nltk
import folium
from folium import plugins
from folium.plugins import HeatMap
from scipy import stats # For in-built method to get PCC
from scipy.stats import pearsonr
from urllib.parse import quote
from matplotlib import rc
import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot, dates
import bokeh 
from bokeh.io import push_notebook, show, output_notebook 
from bokeh.layouts import row
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource
output_notebook()
import plotly.graph_objects as go

#plt.rcParams.update(plt.rcParamsDefault)
#plt.rcParams.update({'figure.figsize': (15, 5)})

Loading BokehJS ...


Shakes and Cracks below the surface: Is the impact the same or different? 

Main concept:

Exploring the inequalities and differences in reporting of Earthquakes occuring in different parts of the world/geographical regions. 
The social, political, and geographical impacts on regions after earthquakes.
Impact on a country’s GDP and financial growth.

More ideas
Despite tectonic plates existing in all parts of the world, there is a strong impact on some countries/regions compared to others.
Regions with the most number of occurrences.
10 highest magnitude earthquakes
Earthquake related to monsoon season, more infrastructural damage?
Time of day, weeks, months, years with most earthquakes, 


In [86]:
from pandas import read_excel

# Loading earthuake dataset 
df_EQ = pd.read_csv("Global_Earthquake_Data.csv")

df_EQ = pd.read_csv("Significant_Earthquake_Dataset_1900-2023.csv")
print(df_EQ.info)

# Loading GDP dataset 
GDP_all = pd.read_csv("gdp_1960_2020.csv")

# Loading Population dataset
Pop = pd.read_csv("word_population.csv")



<bound method DataFrame.info of                            Time                               Place  Latitude  \
0      2023-02-17T09:37:34.868Z        130 km SW of Tual, Indonesia   -6.5986   
1      2023-02-16T05:37:05.138Z       7 km SW of Port-Olry, Vanuatu  -15.0912   
2      2023-02-15T18:10:10.060Z         Masbate region, Philippines   12.3238   
3      2023-02-15T06:38:09.034Z     54 km WNW of Otaki, New Zealand  -40.5465   
4      2023-02-14T13:16:51.072Z         2 km NW of Lele?ti, Romania   45.1126   
...                         ...                                 ...       ...   
37326  1902-01-01T05:20:30.000Z      113 km ESE of Nikolski, Alaska   52.3800   
37327  1901-12-31T09:02:30.000Z       221 km SW of Nikolski, Alaska   51.4500   
37328  1901-12-30T22:34:00.000Z                     south of Alaska   52.0000   
37329  1901-03-03T07:45:00.000Z  12 km NNW of Parkfield, California   36.0000   
37330  1900-10-09T12:25:00.000Z      16 km SW of Old Harbor, Alaska   57.0900

FileNotFoundError: [Errno 2] No such file or directory: 'word_population.csv'

In [4]:
df_EQ.head(5)


,Time,Place,Latitude,Longitude,Depth,Mag,MagType,nst,gap,dmin,...,Updated,Unnamed: 14,Type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-02-17T09:37:34.868Z,"130 km SW of Tual, Indonesia",-6.5986,132.0763,38.615,6.1,mww,119.0,51.0,2.988,...,2023-02-17T17:58:24.040Z,NaN,earthquake,6.41,5.595,0.065,23.0,reviewed,us,us
1,2023-02-16T05:37:05.138Z,"7 km SW of Port-Olry, Vanuatu",-15.0912,167.0294,36.029,5.6,mww,81.0,26.0,0.392,...,2023-02-17T05:41:32.448Z,NaN,earthquake,5.99,6.080,0.073,18.0,reviewed,us,us
2,2023-02-15T18:10:10.060Z,"Masbate region, Philippines",12.3238,123.8662,20.088,6.1,mww,148.0,47.0,5.487,...,2023-02-16T20:12:32.595Z,NaN,earthquake,8.61,4.399,0.037,71.0,reviewed,us,us
3,2023-02-15T06:38:09.034Z,"54 km WNW of Otaki, New Zealand",-40.5465,174.5709,74.320,5.7,mww,81.0,40.0,0.768,...,2023-02-16T06:42:09.738Z,NaN,earthquake,3.68,4.922,0.065,23.0,reviewed,us,us
4,2023-02-14T13:16:51.072Z,"2 km NW of Lele?ti, Romania",45.1126,23.1781,10.000,5.6,mww,132.0,28.0,1.197,...,2023-02-17T09:15:18.586Z,NaN,earthquake,4.85,1.794,0.032,95.0,reviewed,us,us


The  most normal distributed columns are; magnitude, gap, dmin, rms, horizontal error, depth error, magnitude error.

The other appears to have a skewed distribution: Latitude, longitude, depth, nst, magnst.



In [73]:
# We take out a subset of the data we want to look into
df_EQ[df_EQ['Type'].isin(['earthquake'])]
df_EQ_cut = df_EQ[['Updated','Place', 'Latitude', 'Longitude', 'Mag', 'Depth']]
df_EQ_cut.head()

,Updated,Place,Latitude,Longitude,Mag,Depth
0,2023-02-17T17:58:24.040Z,"130 km SW of Tual, Indonesia",-6.5986,132.0763,6.1,38.615
1,2023-02-17T05:41:32.448Z,"7 km SW of Port-Olry, Vanuatu",-15.0912,167.0294,5.6,36.029
2,2023-02-16T20:12:32.595Z,"Masbate region, Philippines",12.3238,123.8662,6.1,20.088
3,2023-02-16T06:42:09.738Z,"54 km WNW of Otaki, New Zealand",-40.5465,174.5709,5.7,74.320
4,2023-02-17T09:15:18.586Z,"2 km NW of Lele?ti, Romania",45.1126,23.1781,5.6,10.000


In [75]:
# We filter the dataset from 2013 - 2023
df_EQ_cut['Time'] = pd.to_datetime(df_EQ_cut['Updated'])
df_EQ_cut['year'] = df_EQ_cut['Time'].dt.year
df_filter = df_EQ_cut[df_EQ_cut['Time'].dt.year.between(2013, 2023)]


In [76]:
df_EQ_cut['year'].value_counts()

2022    33861
2016      948
2020      701
2021      517
2014      514
2019      228
2018      218
2023      204
2015      111
2017       28
2013        1
Name: year, dtype: int64

In [77]:
# Creating a column that specify country

df_filter['Place'] = df_filter['Place'].astype(str)

for index, row in df_filter.iterrows():
    df_filter.loc[index, 'Country'] = row['Place'].split()[-1]

col = list(df_filter.columns)
col.insert(2, col.pop(col.index('Country')))
EQ_filter = df_filter[col]

japan_data = EQ_filter[EQ_filter['Country'] == 'Japan']
print(japan_data)



                        Updated                          Place Country  \
71     2023-01-25T17:20:56.780Z   25 km ESE of Miyazaki, Japan   Japan   
76     2023-01-08T22:02:23.918Z         95 km S of Naze, Japan   Japan   
122    2023-01-28T19:58:14.040Z       85 km SSE of Toba, Japan   Japan   
165    2022-12-10T22:52:33.040Z     26 km SE of Kushima, Japan   Japan   
185    2022-11-26T20:57:37.040Z      166 km W of Itoman, Japan   Japan   
...                         ...                            ...     ...   
37234  2022-04-25T20:36:50.610Z      36 km E of Nobeoka, Japan   Japan   
37241  2022-04-25T20:36:44.935Z  116 km ESE of Katsuura, Japan   Japan   
37249  2022-04-25T20:24:35.887Z      50 km SSW of ?yama, Japan   Japan   
37278  2022-04-26T00:10:30.983Z  217 km ESE of Kamaishi, Japan   Japan   
37283  2022-04-25T20:23:58.797Z     31 km SSW of Hikari, Japan   Japan   

       Latitude  Longitude   Mag  Depth                             Time  year  
71      31.7958   131.6429  5.

In [78]:
EQ_filter.drop(columns=["Place", "Updated"], inplace=True)
EQ_filter['year'] = EQ_filter['Time'].dt.year


import folium
from folium.plugins import HeatMap

# Creates a folium map centered on specific location
M = folium.Map(location=[0.457, 20.456], zoom_start=1)

# Heatmap layer based on the earthquake's latitude and longitude 
heatmap_EQ = [[row['Latitude'], row['Longitude']] for index, row in df_filter.iterrows()]
HeatMap(heatmap_EQ).add_to(M)

# Display the map with the heatmap layer overlaid on top


In [79]:

EQ_filter

,Country,Latitude,Longitude,Mag,Depth,Time,year
0,Indonesia,-6.5986,132.0763,6.10,38.615,2023-02-17 17:58:24.040000+00:00,2023
1,Vanuatu,-15.0912,167.0294,5.60,36.029,2023-02-17 05:41:32.448000+00:00,2023
2,Philippines,12.3238,123.8662,6.10,20.088,2023-02-16 20:12:32.595000+00:00,2023
3,Zealand,-40.5465,174.5709,5.70,74.320,2023-02-16 06:42:09.738000+00:00,2023
4,Romania,45.1126,23.1781,5.60,10.000,2023-02-17 09:15:18.586000+00:00,2023
...,...,...,...,...,...,...,...
37326,Alaska,52.3800,-167.4500,7.00,NaN,2018-06-04 20:43:44+00:00,2018
37327,Alaska,51.4500,-171.0200,7.10,NaN,2018-06-04 20:43:44+00:00,2018
37328,Alaska,52.0000,-160.0000,7.00,NaN,2018-06-04 20:43:44+00:00,2018
37329,California,36.0000,-120.5000,6.40,NaN,2018-06-04 20:43:44+00:00,2018


Regional Earthquake Heatmap: Create an interactive heatmap that shows the frequency and magnitude of earthquakes across different regions of the world. Users could filter the data by year or magnitude, and the heatmap could be color-coded to show the intensity of earthquake activity. This visualization could help illustrate the disparities in earthquake activity and reporting across different regions.

Earthquake Impact Timeline: Create a timeline that shows the social, political, and geographical impacts of earthquakes over time. This could include data on casualties, displacement, economic impact, and political instability in the aftermath of earthquakes. Users could explore the timeline by region or country, and the visualization could include multimedia elements such as photos and videos to help bring the impact of earthquakes to life.

GDP Impact Visualization: Create a visualization that shows the impact of earthquakes on a country's GDP over time. This could include a line graph that shows changes in GDP in the years following an earthquake, as well as a map that shows the geographic distribution of GDP impact. This visualization could help illustrate the long-term economic consequences of earthquakes, particularly in developing countries.

Reporting Disparities Visualization: Create a visualization that shows the disparities in earthquake reporting across different regions of the world. This could include a scatter plot that shows the relationship between earthquake magnitude and the number of reported earthquakes in different regions. The visualization could also include a map that shows which countries have the highest and lowest rates of earthquake reporting. This could help highlight the need for more consistent and reliable earthquake reporting worldwide.

Personal Stories of Impact: In addition to visualizations, consider incorporating personal stories of people impacted by earthquakes. This could include interviews with survivors, rescue workers, and aid workers, as well as personal narratives of people who have experienced the social and economic impacts of earthquakes firsthand. These stories could be integrated into the visualizations and could help bring a human element to the data.

## Impact on a country’s GDP and financial growth. (Mabel)


We are interested in highligting the financial consquences of high impact earthquakes in countries. So we created new subset data for the earthquakes of magnitude 6.0 < based from the magnitude scale taken from an post by Michigan tech: https://www.mtu.edu/geo/community/seismology/learn/earthquake-measure/magnitude/. As


In [80]:
# Making coordinates pair for longitude and langitue 
EQ_filter['Coordinates'] = list(zip(EQ_filter.Longitude, EQ_filter.Latitude))


import plotly.express as px

fig = px.scatter_geo(EQ_filter, lat="Latitude", lon="Longitude", color="Mag",
                     hover_name="Country", size="Mag",animation_frame= "year",
                     projection="natural earth", size_max=8)

fig.show()


Removal of rows with country names, such as "Ridge", "region", "nan", "Rise" as they are designated names for the earthquakes that appear in the oceans as seen on the map or mid-ocean ridge earthquakes. Even though the aftermath of these earthquakes may have impacted some countries. The datavisulisation would be focused on the earthquakes happening directly in the country. 


In [81]:
# Removal of of the rows, designated under the country names "Ridge", "Region", "nan", "Rise"
EQ_filter = EQ_filter[~EQ_filter['Country'].isin(['Ridge', 'region','Rise',np.nan])]

In [82]:

# New dataframe with magnitude above six
EQ_filter_mag = EQ_filter[EQ_filter['Mag'] > 6.0]
print(EQ_filter_mag.size)
print(EQ_filter.size)



86728
257848


In [61]:

# The 20 countries with highest occurence of earthquakes from 2013-2022
contry_count = EQ_filter.groupby('Country').size()
contry_count = contry_count.sort_values(ascending=False)
contry_20 =  contry_count.iloc[:20]

# The 20 countries with the highest occurence of earthquakes above the magnittude 6 from 2013-2022
contry_count_mag = EQ_filter_mag.groupby('Country').size()
contry_count_mag  = contry_count_mag.sort_values(ascending=False)
contry_20_mag  =  contry_count_mag.iloc[:20]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=contry_20.index,
    y=contry_20.values,
    name='Countries',
    marker_color='indianred'
))

fig.update_layout(
    title='Top 20 countries with highest occurences of earthquakes (2013-2023)',
    xaxis_title='Country',
    yaxis_title='No. of earthquakes',
    barmode='group'
)

fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=contry_20_mag.index,
    y=contry_20_mag.values,
    name='Countries',
    marker_color='lightsalmon'
))

#fig2.update_layout(
#    title='Top 20 countries with highest occurences of earthquakes with magnitude above 5.3 (2013-2023)',
#    xaxis_title='Country',
#    yaxis_title='No. of earthquakes'
#)

# combine both plots using a subplot
from plotly.subplots import make_subplots

fig3 = make_subplots(rows=2, cols=1, subplot_titles=('Top 20 countries with highest occurrences of earthquakes (2013-2023)', 'Top 20 countries with earthquakes Above Magnitude 6.0'))
fig3.add_trace(fig.data[0], row=1, col=1)
fig3.add_trace(fig2.data[0], row=2, col=1)

# Defining the axis 
fig3.update_xaxes(title_text="Country", row=2, col=1)
fig3.update_yaxes(title_text="No. of earthquakes", row=1, col=1, title_standoff=10,title_font=dict(size=10))
fig3.update_yaxes(title_text="No. of earthquakes", row=2, col=1, title_standoff=10,title_font=dict(size=10))
fig3.show()

In [68]:
# Number of Earthquakes that occurred from 2013-2023 Japan vs Nepal 

# Filter for earthquakes in Japan and Nepal with magnitude above 6
Japan = EQ_filter[(EQ_filter['Country'] == 'Japan') ]
Indo = EQ_filter[(EQ_filter['Country'] == 'Indonesia') ]

# We group the earthquakes by year and count occurences
japan_count = Japan.groupby('year').size()
Indo_count = Indo.groupby('year').size()

# Create a bar chart showing the earthquake counts for each country

fig = go.Figure()
fig.add_trace(go.Bar(
    x=japan_count.index,
    y=japan_count.values,
    name='Japan',
    marker_color='indianred'
))

fig.add_trace(go.Bar(
    x=Indo_count.index,
    y=Indo_count.values,
    name='Indonesia',
    marker_color='lightsalmon'
))

fig.update_layout(
    title='Earthquakes Occurences in Japan and Nepal (2013-2023)',
    xaxis_title='Year',
    yaxis_title='No. of Earthquakes',
    barmode='group'
)
fig.update_layout(yaxis_type='log')
fig.show()

In [67]:
# Number of Earthquakes that occurred from 2013-2023 Japan vs Nepal 

# Filter for earthquakes in Japan and Nepal with magnitude above 6
Japan = EQ_filter[(EQ_filter['Country'] == 'Japan') ]
Indo = EQ_filter[(EQ_filter['Country'] == 'Indonesia') ]

# We group the earthquakes by year and count occurences
japan_count = Japan.groupby('year').size()
Indo_count = Indo.groupby('year').size()

# Create a bar chart showing the earthquake counts for each country

fig = go.Figure()
fig.add_trace(go.Bar(
    x=japan_count.index,
    y=japan_count.values,
    name='Japan',
    marker_color='indianred'
))

fig.add_trace(go.Bar(
    x=Indo_count.index,
    y=Indo_count.values,
    name='Indonesia',
    marker_color='lightsalmon'
))

fig.update_layout(
    title='Earthquakes Occurences in Japan and Nepal (2013-2023)',
    xaxis_title='Year',
    yaxis_title='No. of Earthquakes',
    barmode='stack'
)
fig.update_layout(yaxis_type='log')
fig.show()

In [ ]:
# Filter for earthquakes in Japan and Nepal with magnitude above 6
Japan = EQ_filter[(EQ_filter['Country'] == 'Japan') ]
Haiti = EQ_filter[(EQ_filter['Country'] == 'Haiti') ]

# We group the earthquakes by year and count occurences
japan_count = Japan.groupby('year').size()
Haiti_count = Haiti.groupby('year').size()

# Create a bar chart showing the earthquake counts for each country
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=japan_count.index,
    y=japan_count.values,
    name='Japan',
    marker_color='indianred'
))

fig.add_trace(go.Bar(
    x=Haiti_count.index,
    y=Haiti_count.values,
    name='Haiti',
    marker_color='lightsalmon'
))

fig.update_layout(
    title='Earthquakes Occurences in Japan and Nepal (2013-2023)',
    xaxis_title='Year',
    yaxis_title='No. of Earthquakes',
    barmode='group'
)
fig.show()